In [1]:
# multivariate multi-step stacked lstm example
import numpy as np
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
import pandas as pd
import pickle as pkl
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [2]:
nthu_train_X = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_train_X", 'rb'))
nthu_train_y = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_train_y", 'rb'))

nthu_test_X = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_test_X", 'rb'))
nthu_test_y = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_test_y", 'rb'))

In [3]:
nthu_train_y = nthu_train_y.mean(axis=1)
nthu_test_y = nthu_test_y.mean(axis=1)

In [4]:
nthu_train_y_scaled = tf.keras.activations.sigmoid(tf.constant(nthu_train_y, dtype = tf.float32))
nthu_train_y_scaled = nthu_train_y_scaled.numpy()

In [5]:
nthu_test_y_scaled = tf.keras.activations.sigmoid(tf.constant(nthu_test_y, dtype = tf.float32))
nthu_test_y_scaled = nthu_test_y_scaled.numpy()

In [6]:
n_steps_in, n_features, n_steps_out= 150, 8, 1

# define model
model = Sequential()
model.add(Bidirectional(LSTM(100, activation='tanh'), input_shape=(n_steps_in, n_features)))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(nthu_train_X, np.ravel(nthu_train_y_scaled), batch_size=8, epochs=32, verbose=2)

Epoch 1/32
52620/52620 - 658s - loss: 0.0184
Epoch 2/32
52620/52620 - 661s - loss: 0.0098
Epoch 3/32
52620/52620 - 658s - loss: 0.0060
Epoch 4/32
52620/52620 - 657s - loss: 0.0043
Epoch 5/32
52620/52620 - 661s - loss: 0.0029
Epoch 6/32
52620/52620 - 657s - loss: 0.0022
Epoch 7/32
52620/52620 - 656s - loss: 0.0016
Epoch 8/32
52620/52620 - 656s - loss: 0.0013
Epoch 9/32
52620/52620 - 657s - loss: 0.0011
Epoch 10/32
52620/52620 - 665s - loss: 9.0502e-04
Epoch 11/32
52620/52620 - 663s - loss: 7.9550e-04
Epoch 12/32
52620/52620 - 660s - loss: 7.0833e-04
Epoch 13/32
52620/52620 - 656s - loss: 6.7532e-04
Epoch 14/32
52620/52620 - 656s - loss: 6.2202e-04
Epoch 15/32
52620/52620 - 656s - loss: 5.3928e-04
Epoch 16/32
52620/52620 - 660s - loss: 5.2704e-04
Epoch 17/32
52620/52620 - 657s - loss: 4.7184e-04
Epoch 18/32
52620/52620 - 660s - loss: 4.5117e-04
Epoch 19/32
52620/52620 - 663s - loss: 4.3531e-04
Epoch 20/32
52620/52620 - 664s - loss: 4.3302e-04
Epoch 21/32
52620/52620 - 657s - loss: 0.0017

In [7]:
# demonstrate prediction
yhat = model.predict(nthu_test_X, verbose=1)
print(yhat)

4101/4101 [==============================] - 25s 6ms/step
[[0.7423377 ]
 [0.6883625 ]
 [0.7156741 ]
 ...
 [0.76782435]
 [0.77155197]
 [0.6880797 ]]


In [8]:
def logit(x):
    """ Computes the logit function, i.e. the logistic sigmoid inverse. """
    if (x<=0.5):
        return float(0)
    if (x>=0.880797):
        return float(2)
    return float(- tf.math.log(1. / x - 1.))

In [9]:
yhat_inversed = np.array([logit(x) for x in yhat])

In [10]:
mean_squared_error(yhat_inversed,nthu_test_y)

0.9583501190462538

In [11]:
model.save("lstm_bidirectional.h5")